In [159]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import requests
from tqdm import tqdm
import numpy as np
import json

In [55]:
bkms_reactions = pd.read_csv('bkms/1Sep2023_bkms-mapped.txt',sep='\t')

In [101]:
kegg_rids = bkms_reactions['Reaction_ID_KEGG'].drop_duplicates().tolist()

In [69]:
len(bkms_reactions['Reaction_ID_KEGG'].drop_duplicates().tolist())

5862

In [88]:
class KeggPage():
    def __init__(self, url):
        res = requests.get(url)
        self.soup = BeautifulSoup(res.text, 'html.parser')
        self.get_other_dbs()
        
    def get_other_dbs(self):
        for tr in self.soup.find_all('tr'):
            if 'Other DBs' in tr.text:
                self.other_dbs = tr.text
    
#     def get_other_dbs_ref(self):
        

In [89]:
kp = KeggPage('https://www.genome.jp/entry/R03024')

In [90]:
kp.other_dbs

'Other DBs\nRHEA:\xa021667'

In [80]:
[x.text if x is not None else x for x in kp.soup.find_all('tr')]

['\n\xa0\xa0\xa0REACTION: R03024Help\n\n\n\nEntry\nR03024\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Reaction\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\n\nName\n4-nitrophenyl phosphate phosphohydrolase\n\nDefinition\n4-Nitrophenyl phosphate + H2O <=> 4-Nitrophenol + Orthophosphate\n\nEquation\nC03360 + C00001 <=> C00870 + C00009\n\n\n\nReaction class\nRC00151\xa0\xa0C00870_C03360\nEnzyme\n3.1.3.1\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa03.1.3.2\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa03.1.3.41\n\nPathway\nrn00627\xa0\xa0Aminobenzoate degradationrn01120\xa0\xa0Microbial metabolism in diverse environments\nBrite\nEnzymatic reactions [BR:br08201]\n\xa03. Hydrolase reactions\n\xa0\xa03.1\xa0\xa0Acting on ester bonds\n\xa0\xa0\xa03.1.3\xa0\xa0Phosphoric-monoester hydrolases\n\xa0\xa0\xa0\xa03.1.3.1\n\xa0\xa0\xa0\xa0\xa0R03024\xa0\xa04-Nitrophenyl phosphate + H2O <=> 4-Nitropheno

In [87]:
'Other DBs' in [x for x in kp.soup.find_all('tr')][-2].text

True

In [54]:
for p in [x for x in kp.soup.find_all('td', class_='td{}'.format(kp.other_dbs_idx))]:
    print ([y.text for y in p.find_all('table')])

['R03024\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Reaction\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\n']
[]
[]
[]
[]
['RHEA:\xa021667']


In [53]:
[x for x in kp.soup.find_all('td', class_='td{}'.format(kp.other_dbs_idx))][-1].find_all('table')

[<table class="w1"><tr><td class="vtop pd0"><span class="nowrap">RHEA: </span></td><td><a href="https://www.rhea-db.org/reaction?id=21667">21667</a></td></tr></table>]

In [91]:
kps = [KeggPage('https://www.genome.jp/entry/'+r.split(',')[0]) for r in bkms_reactions['Reaction_ID_KEGG'].drop_duplicates().tolist()[:100]]

In [99]:
for k in kps:
    try:
        print (k.other_dbs)
    except:
        print ('failed')

Other DBs
RHEA: 10575
Other DBs
RHEA: 10915
Other DBs
RHEA: 16088
Other DBs
RHEA: 19300
Other DBs
RHEA: 23759
Other DBs
RHEA: 10039
Other DBs
RHEA: 23587
Other DBs
RHEA: 23679
Other DBs
RHEA: 30574
Other DBs
RHEA: 16220
Other DBs
RHEA: 10443
Other DBs
RHEA: 21007
Other DBs
RHEA: 14304
Other DBs
RHEA: 12559
Other DBs
RHEA: 13724
Other DBs
RHEA: 16084
Other DBs
RHEA: 21159
Other DBs
RHEA: 66375
Other DBs
RHEA: 11083
Other DBs
RHEA: 12415
Other DBs
RHEA: 15780
Other DBs
RHEA: 24583
Other DBs
RHEA: 16496
Other DBs
RHEA: 13288
Other DBs
RHEA: 22975
Other DBs
RHEA: 22971
Other DBs
RHEA: 10375
Other DBs
RHEA: 10299
Other DBs
RHEA: 10303
Other DBs
RHEA: 14860
Other DBs
RHEA: 19844
Other DBs
RHEA: 56859
Other DBs
RHEA: 26565
Other DBs
RHEA: 10187
Other DBs
RHEA: 12748
Other DBs
RHEA: 23111
Other DBs
RHEA: 10363
Other DBs
RHEA: 12375
Other DBs
RHEA: 13108
Other DBs
RHEA: 19540
Other DBs
RHEA: 20963
Other DBs
RHEA: 21667
Other DBs
RHEA: 17292
Other DBs
RHEA: 10687
Other DBs
RHEA: 13560
Other DBs


In [68]:
for i in range(10):
    kp = kps[i]
    print (i)
    for p in [x for x in kp.soup.find_all('td', class_='td{}'.format(kp.other_dbs_idx))]:
        print ([y.text for y in p.find_all('table')])

0
['R03031\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Reaction\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\n']
[]
[]
[]
[]
['RHEA:\xa010575']
1
[]
[]
['RC00096\xa0\xa0C00099_C00864', 'RC00141\xa0\xa0C00522_C00864']
['rn00410\xa0\xa0beta-Alanine metabolism', 'rn00770\xa0\xa0Pantothenate and CoA biosynthesis', 'rn01100\xa0\xa0Metabolic pathways', 'rn01110\xa0\xa0Biosynthesis of secondary metabolites', 'rn01240\xa0\xa0Biosynthesis of cofactors']
[]
['RHEA:\xa010915']
2
['R04573\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Reaction\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\n']
[]
[]
[]
[]
['RHEA:\xa016088']
3
['R01164\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Reaction\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa

In [315]:
def get_genes_for_kegg_rid (krid):
    kegg_gene_url = 'https://www.genome.jp/dbget-bin/get_linkdb?-t+genes+rn:'
    url = kegg_gene_url+krid
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    return [a.get('href') for a in soup.find_all('a') if a.get('href') and 'entry' in a.get('href')]

In [314]:
get_genes_for_kegg_rid('R04132')

<!doctype html>
<html>
  <head>
    <meta charset="utf-8">
    <title>
KEGG REACTION R04132    </title>
    <link href="/css/gn2.css?1700434570" rel="stylesheet" type="text/css">
    <style>
      pre{font-family:monospace}
      #gotop {display:none;position:fixed;bottom:50px;left:50rem;cursor:pointer;width:30px;}
    </style>
    <script src="/js/linkdb_scr.js?1700434570"></script>
    <script>
      onload = function(){
        var obj = document.getElementById("gotop") ? document.getElementById("gotop") : {};
        if(!obj){return;}
        obj.addEventListener("click",function(e){e.preventDefault();scrt(this);},false);
      };
      onscroll = function(){onscr();};
    </script>
  </head>
  <body>
  <a id="top"></a>
<table width="680">
<tr>
  <td width="180" align="center">
    <a href="https://www.genome.jp/"><img src="/Fig/gntop2.gif" alt="GenomeNet icon" border="0"></a>
  </td>
  <td>
    <h2 class="title2">LinkDB Search</h2>
  </td>
</tr>
</table>
<br>
Mode: Single Entry to

['/entry/aaa:Acav_3009',
 '/entry/aaa:Acav_4171',
 '/entry/aacd:LWP59_17280',
 '/entry/aaeg:RA224_12250',
 '/entry/aagi:NCTC2676_1_02912',
 '/entry/aai:AARI_06160',
 '/entry/aak:AA2016_5674',
 '/entry/aaq:AOC05_17470',
 '/entry/aaqu:D3M96_17660',
 '/entry/aarc:G127AT_03995',
 '/entry/aau:AAur_3913',
 '/entry/aau:AAur_pTC20093',
 '/entry/aav:Aave_4269',
 '/entry/aay:WYH_01857',
 '/entry/abau:IX87_20465',
 '/entry/abaz:P795_8040',
 '/entry/abk:LX00_09725',
 '/entry/abom:D7I45_02260',
 '/entry/abou:ACBO_11690',
 '/entry/acao:NF551_03975',
 '/entry/ach:Achl_3612',
 '/entry/achb:DVB37_11180',
 '/entry/achb:DVB37_13620',
 '/entry/achb:DVB37_16885',
 '/entry/acho:H4P35_15075',
 '/entry/achr:C2U31_20035',
 '/entry/achr:C2U31_24580',
 '/entry/acim:GT370_09710',
 '/entry/aciu:A3K93_02920',
 '/entry/acom:CEW83_16445',
 '/entry/acry:AC20117_13400',
 '/entry/adi:B5T_01363',
 '/entry/adi:B5T_03296',
 '/entry/adk:Alide2_0518',
 '/entry/adt:APT56_13965',
 '/entry/adt:APT56_18395',
 '/entry/ady:HLG70_0

In [317]:
kegg_rids_to_genes['R04132']

[]

In [136]:
get_genes_for_kegg_rid(kegg_rids[0])

['/entry/aalt:CC77DRAFT_1022132',
 '/entry/abe:ARB_01878',
 '/entry/ache:ACHE_30591S',
 '/entry/act:ACLA_020400',
 '/entry/afm:AFUA_1G14950',
 '/entry/afv:AFLA_004052',
 '/entry/ago:AGOS_AFR440C',
 '/entry/aje:HCAG_02422',
 '/entry/aluc:AKAW2_50743A',
 '/entry/amus:LMH87_002178',
 '/entry/ang:An03g03460',
 '/entry/ani:AN0836.2',
 '/entry/aor:AO090005001234',
 '/entry/apuu:APUU_50980S',
 '/entry/arab:EKO05_0007074',
 '/entry/bbel:109465946',
 '/entry/bbrx:BRETT_004030',
 '/entry/bcom:BAUCODRAFT_282374',
 '/entry/bfu:BCIN_09g06500',
 '/entry/bgh:BDBG_08395',
 '/entry/bgt:106075236',
 '/entry/bnn:FOA43_001988',
 '/entry/bor:COCMIDRAFT_94913',
 '/entry/bsc:COCSADRAFT_41756',
 '/entry/bze:COCCADRAFT_84782',
 '/entry/cal:CAALFM_CR09790WA',
 '/entry/caur:CJI96_0004650',
 '/entry/cbet:CB0940_02807',
 '/entry/cdu:CD36_34940',
 '/entry/cfj:CFIO01_11235',
 '/entry/cgr:CAGL0H04125g',
 '/entry/chig:CH63R_07224',
 '/entry/chig:CH63R_08599',
 '/entry/cim:CIMG_03293',
 '/entry/clu:CLUG_02182',
 '/entr

In [144]:
unique_kegg_rids =(','.join([str(x) for x in kegg_rids])).split(',')

In [204]:
unique_kegg_rids

['R03031',
 'R02473',
 'R04573',
 'R01164',
 'R02715',
 'R02877',
 'R02237',
 'R02788',
 'R07644',
 'R02526',
 'R01519',
 'R03156',
 'R02992',
 'R02035',
 'R03641',
 'R04317',
 'R01748',
 'R00841',
 'R07298',
 'R08658',
 'R01569',
 'R01511',
 'R03496',
 'R02866',
 'R02648',
 'R00894',
 'R01173',
 'R01172',
 'R02450',
 'R01063',
 'R01061',
 'R03095',
 'R03015',
 'R00544',
 'R07263',
 'R02991',
 'R03708',
 'R02714',
 'R03692',
 'R03893',
 'R06838',
 'R01043',
 'R02167',
 'R03372',
 'R03024',
 'R00763',
 'R02731',
 'R02585',
 'R00497',
 'R04151',
 'R03962',
 'R02268',
 'R01633',
 'R05150',
 'R04591',
 'R02451',
 'R02786',
 'R03193',
 'R02783',
 'R04208',
 'R02301',
 'R00804',
 'R00805',
 'R06211',
 'R03371',
 'R07584',
 'R03182',
 'R01230',
 'R04695',
 'R03402',
 'R01585',
 'R04144',
 'R07641',
 'R01954',
 'R04713',
 'R03832',
 'R03835',
 'R01053',
 'R04716',
 'R04068',
 'R03948',
 'R05180',
 'R05149',
 'R00344',
 'R01859',
 'R04138',
 'R03494',
 'R05275',
 'R00304',
 'R00947',
 'R02778',

In [148]:
from joblib import Parallel, delayed

In [318]:
kegg_genes = [get_genes_for_kegg_rid(x) for x in tqdm(unique_kegg_rids)]

  1%|▊                                                                                                               | 43/6019 [01:33<2:36:41,  1.57s/it]
KeyboardInterrupt



In [207]:
kegg_gene_ids = [[y.split('/')[-1] for y in x] for x in kegg_genes]

In [214]:
#UNCOMMENT TO SAVE
# with open('kegg_rids_to_genes.json','w') as f:
#     json.dump(dict(zip(unique_kegg_rids, kegg_gene_ids)), f)


In [287]:
def parse_kegg_header (kegg_header):
    if '[' in kegg_header:
        pattern = r'>(\w+:\w+)\s+(\w+)\s+(.*)\s+\['
    else:
        pattern = r'>(\w+:\w+)\s+(\w+)\s+(.*)'
    # Use re.match to apply the pattern to the input string
    match = re.match(pattern, kegg_header)

    if match:
        # Extract matched groups
        gene_id = match.group(1)
        id_value = match.group(2)
        common_name = match.group(3)

        # Create the desired output
        result = {
            'gene_id': gene_id,
            'id': id_value,
            'common_name': common_name,
        }
        ec_pattern = r'\[EC:(.*)\]'
        ec_match = re.findall(ec_pattern, kegg_header)
        if len(ec_match):
            ec_numbers = ec_match[0]
            result['ec_numbers'] = ec_numbers.split(' ')
        return result
    else:
        print("No match found.")
        return None
    
def parse_kegg_fasta(kegg_fasta):
    split = kegg_fasta.split('\n')
    header = split[1]
    results = parse_kegg_header(header)
    results['sequence'] = ''.join(split[2:])
    return results

input_string = ">iho:Igni_0595 K14534 4-hydroxybutyryl-CoA dehydratase / vinylacetyl-CoA-Delta-isomerase [EC:4.2.1.120 5.3.3.3]"
parse_kegg_header(input_string)

{'gene_id': 'iho:Igni_0595',
 'id': 'K14534',
 'common_name': '4-hydroxybutyryl-CoA dehydratase / vinylacetyl-CoA-Delta-isomerase',
 'ec_numbers': ['4.2.1.120', '5.3.3.3']}

In [274]:
input_string = ">iho:Igni_0595 K14534 4-hydroxybutyryl-CoA dehydratase / vinylacetyl-CoA-Delta-isomerase [EC:4.2.1.120 5.3.3.3]"

pattern = r'\[EC:(.*)\]'

match = re.search(pattern, input_string)

In [303]:
kegg_rids_to_genes = dict(zip(unique_kegg_rids, kegg_gene_ids))

In [220]:
kegg_rids_to_sequence = {}

In [233]:
def retrieve_sequence_for_first_gene(list_of_genes):
    list_of_genes = [x for x in list_of_genes if x]
    if list_of_genes:
        try:
            gene_to_query = list_of_genes[0]
            res = requests.get('https://www.genome.jp/entry/-f+-n+a+'+gene_to_query)
            soup = BeautifulSoup(res.text, 'html.parser')
            return parse_kegg_fasta(soup.find_all('pre')[0].text)
        except IndexError:
            return None
        except TypeError:
            print(list_of_genes)
            return None
        

In [288]:
kegg_sequences = Parallel(n_jobs=16)(delayed(retrieve_sequence_for_first_gene)(x) for x in tqdm(list(kegg_rids_to_genes.values())))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6019/6019 [00:43<00:00, 137.42it/s]


In [304]:
kegg_rids_to_sequences = dict(zip(unique_kegg_rids, kegg_sequences))

In [309]:
{r for r,s in kegg_rids_to_sequences.items() if s is None}

{'R07370',
 'R07532',
 'R05808',
 'R03333',
 'R05168',
 'R02709',
 'R05219',
 'R02661',
 'R06933',
 'R05831',
 'R08874',
 'R08666',
 'R05759',
 'R09134',
 'R08370',
 'R05199',
 'R03964',
 'R07144',
 'R03934',
 'R03088',
 'R02810',
 'R08340',
 'R03627',
 'R07605',
 'R05124',
 'R07865',
 'R00156',
 'R02691',
 'R00364',
 'R08821',
 'R05398',
 'R05888',
 'R08069',
 'R08562',
 'R03292',
 'R06347',
 'R08879',
 'R07360',
 'R05426',
 'R06545',
 'R03879',
 'R07838',
 'R09090',
 'R08869',
 'R08788',
 'R04990',
 'R00004',
 'R07942',
 'R06743',
 'R05450',
 'R07799',
 'R02989',
 'R04237',
 'R03763',
 'R01410',
 'R08312',
 'R06624',
 'R02310',
 'R08560',
 'R07013',
 'R04132',
 'R03760',
 'R07145',
 'R05648',
 'R00836',
 'R03304',
 'R09272',
 'R03959',
 'R09282',
 'R04030',
 'R02923',
 'R05956',
 'R06856',
 'R06092',
 'R07354',
 'R02655',
 'R03374',
 'R08938',
 'R02531',
 'R08392',
 'R06785',
 'R01036',
 'R03792',
 'R04150',
 'R00025',
 'R04880',
 'R08274',
 'R06196',
 'R06449',
 'R06371',
 'R07414',

In [199]:
len(kegg_rids_to_genes)

6019

In [203]:
len([r for r in kegg_genes if not len(r)])

5585

In [230]:
if None:
    print ('hi')

In [312]:
retrieve_sequence_for_first_gene(kegg_rids_to_genes['R04132'])

In [313]:
kegg_rids_to_genes['R04132']

[]

In [294]:
get_genes_for_kegg_rid('R07370')

[]

In [306]:
kegg_rids_to_genes['R00672']

['aacx:DEACI_0852',
 'aacx:DEACI_2039',
 'aacx:DEACI_3360',
 'aacx:DEACI_4184',
 'aah:CF65_01171',
 'aaj:BOQ57_13355',
 'aaj:BOQ57_18740',
 'aak:AA2016_2455',
 'aak:AA2016_5285',
 'aal:EP13_18780',
 'aala:IGS74_12915',
 'aall:I6G90_08240',
 'aall:I6G90_12130',
 'aalm:LUX29_14795',
 'aaln:Q3V95_08515',
 'aamy:GFC30_266',
 'aao:ANH9381_0863',
 'aar:Acear_1717',
 'aat:D11S_0520',
 'aaus:EP12_19560',
 'aaw:AVL56_19350',
 'aaz:ADJ80_03080',
 'abaa:IX88_13550',
 'abab:BJAB0715_01556',
 'abab:BJAB0715_02584',
 'abad:ABD1_21980',
 'abae:CL176_09490',
 'abaj:BJAB0868_02443',
 'abal:ABLAC_13040',
 'abal:ABLAC_21810',
 'abat:CFX1CAM_0751',
 'abau:IX87_06990',
 'abaz:P795_5955',
 'abb:ABBFA_01246',
 'abc:ACICU_02402',
 'aber:BSR55_21015',
 'abes:IU367_03535',
 'abes:IU367_08670',
 'abg:Asbog_00107',
 'abh:M3Q_2671',
 'abj:BJAB07104_02561',
 'abk:LX00_11875',
 'abn:AB57_2538',
 'abr:ABTJ_01319',
 'abut:Ami103574_00565',
 'abut:Ami103574_07050',
 'abut:Ami103574_14465',
 'abx:ABK1_1287',
 'aby:ABAYE

In [301]:
kegg_rids_to_genes['R00015']

In [302]:
kegg_rids_to_genes

{'R03031': {'gene_id': 'aacx:DEACI_0207',
  'id': 'K14534',
  'common_name': '4-hydroxybutyryl-CoA dehydratase / vinylacetyl-CoA-Delta-isomerase',
  'ec_numbers': ['4.2.1.120', '5.3.3.3'],
  'sequence': 'MKTAAQYEESLRRLNLKVYLQGERVENPVDHPIIRPSMNSVKTTYALAEDPLYQDLMTATSHLTGKKINRFCHLHQSTDDLVKKIKMQRLLGQKTAACFQRCVGMDAINAVDSVTFEMDQKLGTKYHERFTQFLLKMQEEDWTVDGAMTDPKGDRGLSPSQQTDPDLYVHVTERTDDGIVVRGAKAHQTGALNSHWILVMPTSTMRQEDADYALSFVAPADGKGIFYIYGRQSSDTRKLEGGDIDVGNKQFGGHEALMVFDDVFIPWENVFMCGEYEFSGVLVERFAGYHRQSYGGCKVGVGDVLIGAAAVAADYNGAAKASHIKDKLVEMTHLNETLYACGIACSAEGHKTASGTYLINMLLANVTKQNVTRFPYEIARLAEDIAGGLMVTMPSEKDLRHPEIGKVVEKYFRGVASVPTEDRMRILRLIENLTLGTAAVGYRTESMHGAGSPQAQRIMIARQSNPERKKELAKAIAGIGTN'},
 'R02473': {'gene_id': 'aaa:Acav_3761',
  'id': 'K01918',
  'common_name': 'pantoate--beta-alanine ligase',
  'ec_numbers': ['6.3.2.1'],
  'sequence': 'MLIAHSIADLRSALASRGRPAFVPTMGNLHEGHLSLVRQARALGDVSVASIFVNRLQFLPHEDFDSYPRTWEADRAQLEAAGCDILFAPRETDLYPQAQTFKVQPDPLLADLLEGQFRPGFFTGVCTVVMKLFSAVFGTTGGGTALFGKKDYQQLMVI